### Natural break

In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.preprocessing import StandardScaler

# Load the data
df = pd.read_csv('../data/api/APIMS-monthly.csv')

# 1. JENKS NATURAL BREAKS (Fisher-Jenks algorithm)
# Best for finding natural groupings in data
def jenks_breaks(data, n_classes):
    """
    Compute Jenks Natural Breaks classification
    """
    data = np.array(sorted(data))
    n = len(data)
    
    # Initialize matrices
    mat1 = np.zeros((n, n_classes))
    mat2 = np.zeros((n, n_classes))
    
    for i in range(n):
        mat1[i, 0] = 1
        mat2[i, 0] = 0
        for j in range(1, n_classes):
            mat1[i, j] = float('inf')
    
    # Compute variance for each subset
    for l in range(2, n + 1):
        s1 = s2 = w = 0
        for m in range(1, l + 1):
            i3 = l - m + 1
            val = data[i3 - 1]
            
            s2 += val * val
            s1 += val
            w += 1
            
            v = s2 - (s1 * s1) / w
            i4 = i3 - 1
            
            if i4 != 0:
                for j in range(2, n_classes + 1):
                    if mat1[l - 1, j - 1] >= v + mat1[i4 - 1, j - 2]:
                        mat1[l - 1, j - 1] = v + mat1[i4 - 1, j - 2]
                        mat2[l - 1, j - 1] = i3
    
    # Extract breaks
    k = n
    kclass = [0] * (n_classes + 1)
    kclass[n_classes] = data[-1]
    
    for j in range(n_classes, 1, -1):
        id_val = int(mat2[k - 1, j - 1] - 2)
        kclass[j - 1] = data[id_val]
        k = int(mat2[k - 1, j - 1] - 1)
    
    return kclass[1:]

# 2. K-MEANS CLUSTERING
def kmeans_breaks(data, n_classes):
    """
    Use K-means clustering to find breaks
    """
    kmeans = KMeans(n_clusters=n_classes, random_state=42)
    data_reshaped = data.values.reshape(-1, 1)
    kmeans.fit(data_reshaped)
    
    # Get cluster centers and sort them
    centers = sorted(kmeans.cluster_centers_.flatten())
    
    # Breaks are midpoints between centers
    breaks = [(centers[i] + centers[i+1]) / 2 for i in range(len(centers)-1)]
    breaks.append(data.max())
    
    return breaks

# 3. QUANTILE-BASED (Equal Count)
def quantile_breaks(data, n_classes):
    """
    Equal count breaks using quantiles
    """
    quantiles = np.linspace(0, 1, n_classes + 1)[1:]
    breaks = [data.quantile(q) for q in quantiles]
    return breaks

# 4. EQUAL INTERVAL
def equal_interval_breaks(data, n_classes):
    """
    Equal interval breaks
    """
    min_val = data.min()
    max_val = data.max()
    interval = (max_val - min_val) / n_classes
    breaks = [min_val + interval * (i + 1) for i in range(n_classes)]
    return breaks

# 5. STANDARD DEVIATION
def std_breaks(data, n_classes):
    """
    Standard deviation breaks
    """
    mean = data.mean()
    std = data.std()
    
    # Create breaks at mean ± std intervals
    breaks = []
    for i in range(-(n_classes // 2), (n_classes // 2) + 1):
        if i != -(n_classes // 2):  # Skip first break
            breaks.append(mean + i * std)
    
    # Adjust to data range
    breaks = [max(data.min(), min(data.max(), b)) for b in breaks]
    return sorted(set(breaks))

# Compare all methods
n_classes = 6
adt_data = df['aqi']

print("="*80)
print(f"OPTIMAL BREAK POINT ANALYSIS (n_classes={n_classes})")
print("="*80)

methods = {
    'Jenks Natural Breaks': jenks_breaks(adt_data, n_classes),
    'K-Means Clustering': kmeans_breaks(adt_data, n_classes),
    'Quantile (Equal Count)': quantile_breaks(adt_data, n_classes),
    'Equal Interval': equal_interval_breaks(adt_data, n_classes),
    'Standard Deviation': std_breaks(adt_data, n_classes)
}

for method_name, breaks in methods.items():
    print(f"\n{method_name}:")
    print(f"  Breaks: {[f'{b:,.0f}' for b in breaks]}")
    
    # Calculate how many observations in each class
    bins = [adt_data.min() - 1] + breaks
    labels = [f'Class {i+1}' for i in range(len(breaks))]
    adt_data_classified = pd.cut(adt_data, bins=bins, labels=labels, include_lowest=True)
    counts = adt_data_classified.value_counts().sort_index()
    
    print(f"  Distribution:")
    for label, count in counts.items():
        pct = (count / len(adt_data)) * 100
        print(f"    {label}: {count:>5} ({pct:>5.1f}%)")

# 6. GOODNESS OF VARIANCE FIT (GVF)
# Measures how well the classification preserves variance
def calculate_gvf(data, breaks):
    """
    Calculate Goodness of Variance Fit
    Higher is better (closer to 1.0)
    """
    bins = [data.min() - 1] + breaks
    classes = pd.cut(data, bins=bins, include_lowest=True)
    
    # Total sum of squared deviations
    sdam = ((data - data.mean()) ** 2).sum()
    
    # Within-class sum of squared deviations
    sdcm = 0
    for class_label in classes.unique():
        if pd.notna(class_label):
            class_data = data[classes == class_label]
            sdcm += ((class_data - class_data.mean()) ** 2).sum()
    
    gvf = (sdam - sdcm) / sdam
    return gvf

print("\n" + "="*80)
print("GOODNESS OF VARIANCE FIT (GVF) - Higher is Better")
print("="*80)

for method_name, breaks in methods.items():
    gvf = calculate_gvf(adt_data, breaks)
    print(f"{method_name:.<50} GVF = {gvf:.4f}")

OPTIMAL BREAK POINT ANALYSIS (n_classes=6)

Jenks Natural Breaks:
  Breaks: ['113', '117', '120', '134', '135', '150']
  Distribution:
    Class 1: 10115 (100.0%)
    Class 2:     1 (  0.0%)
    Class 3:     1 (  0.0%)
    Class 4:     1 (  0.0%)
    Class 5:     1 (  0.0%)
    Class 6:     1 (  0.0%)

K-Means Clustering:
  Breaks: ['27', '37', '46', '55', '76', '150']
  Distribution:
    Class 1:  1281 ( 12.7%)
    Class 2:  2316 ( 22.9%)
    Class 3:  2735 ( 27.0%)
    Class 4:  2470 ( 24.4%)
    Class 5:  1234 ( 12.2%)
    Class 6:    84 (  0.8%)

Quantile (Equal Count):
  Breaks: ['30', '36', '42', '47', '53', '150']
  Distribution:
    Class 1:  1688 ( 16.7%)
    Class 2:  1685 ( 16.7%)
    Class 3:  1690 ( 16.7%)
    Class 4:  1683 ( 16.6%)
    Class 5:  1688 ( 16.7%)
    Class 6:  1686 ( 16.7%)

Equal Interval:
  Breaks: ['34', '57', '80', '103', '127', '150']
  Distribution:
    Class 1:  2710 ( 26.8%)
    Class 2:  6445 ( 63.7%)
    Class 3:   903 (  8.9%)
    Class 4:    47 (

### Top k statistic

In [ ]:
import pandas as pd

# load the cleaned data
melted_df = pd.read_csv('../data/traffic/adt_data.csv')

# find top k busiest roads by year
k = 25
top_k_by_year = melted_df.groupby('Year').apply(lambda x: x.nlargest(k, 'ADT')).reset_index(drop=True)

top_k_by_year

In [ ]:
# find all unique stations in the top k list
unique_stations = top_k_by_year['Location'].unique()
unique_stations